In [1]:
using TensorFlow
tf_version()

v"1.10.0"

# Save and Restore

The `train.Saver` type provides methods to save and restore models. Follow this guide to learn how to do this, or checkout the [Saving and restoring variable values](http://malmaud.github.io/TensorFlow.jl/latest/saving.html) section of the official TensorFlow.jl documentation.

**⋆Note**: Because TensorFlow.jl does not have a `reset_default_graph()` method implemented, make sure to restart your Julia kernel at the indicated cells to reset the TensorFlow environment. 

## Save and restore variables

TensorFlow [Variables](TensorFlow%20Guide%20-%20Variables.ipynb) are the best way to represent shared, persistent state
manipulated by your program. The `train.Saver` constructor adds `save` and
`restore` ops to the graph for all, or a specified list, of the variables in the
graph.  The `Saver` object provides methods to run these ops, specifying paths
for the checkpoint files to write to or read from.

`Saver` restores all variables already defined in your model. If you're
loading a model without knowing how to build its graph (for example, if you're
writing a generic program to load models), then read the
[Overview of saving and restoring models](#models) section
later in this document.

TensorFlow saves variables in binary *checkpoint files* that map variable
names to tensor values.

**!Caution**: TensorFlow model files are code. Be careful with untrusted code.
See [Using TensorFlow Securely](https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md)
for details.

### Save variables

Create a `Saver` with `train.Saver()` to manage all variables in the
model. For example, the following snippet demonstrates how to call the
`save` method to save variables to checkpoint files:

In [4]:
# Create some variables.
@tf v1 = get_variable([3], Float64; initializer=ConstantInitializer(0.0))
@tf v2 = get_variable([5], Float64; initializer=ConstantInitializer(0.0))

inc_v1 = assign(v1, v1 + 1)
dec_v2 = assign(v2, v2 - 1)

# Add ops to save and restore all the variables.
saver = train.Saver()

# Later, launch the model, initialize the variables, do some work, and save the variables to disk.
sess = Session()
run(sess, global_variables_initializer())
# Do some work with the model.
run(sess, inc_v1)
run(sess, dec_v2)
# Save the variables to disk.
train.save(saver, sess, "model.jld")
println("Model saved")
close(sess)

Model saved


2018-10-05 09:38:55.748325: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA


**⋆Note**: TensorFlow.jl uses the [JDL2](https://github.com/simonster/JLD2.jl) format, which is different from the Python format. As such, currently it is not possible to share parameter between Julia and Python-trained models. Refer to the [official documentation - Limitations](http://malmaud.github.io/TensorFlow.jl/latest/index.html#Limitations-1) for more info.

### Restore variables

The `train.Saver` object not only saves variables to checkpoint files, it
also restores variables. Note that when you restore variables you do not have
to initialize them beforehand. For example, the following snippet demonstrates
how to call the `train.restore` method to restore variables from the
checkpoint files:

In [1]:
# !!! Restart your Julia kernel here !!!
using TensorFlow

# Create some variables.
@tf v1 = get_variable([3], Float64)
@tf v2 = get_variable([5], Float64)

# Add ops to save and restore all the variables.
saver = train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
sess = Session()
# Restore variables from disk.
train.restore(saver, sess, "model.jld")
println("Model restored")
# Check the values of the variables
println("v1 : ", run(sess, v1))
println("v2 : ", run(sess, v2))
close(sess)

Model restored


2018-10-05 09:39:23.936891: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA


v1 : [1.0, 1.0, 1.0]
v2 : [-1.0, -1.0, -1.0, -1.0, -1.0]


### Choose variables to save and restore

If you do not pass any arguments to `train.Saver()`, the saver handles all variables in the graph. It is sometimes useful to only save or restore a subset of the variables
used by a model.  For example, you may have trained a neural net with five
layers, and you now want to train a new model with six layers that reuses the
existing weights of the five trained layers. You can use the saver to restore
the weights of just the first five layers.

You can easily specify the variables to save or load by passing to the `var_list` keyword argument of the `train.Saver()` constructor a list of variables.

Continuing from the save/restore examples shown earlier (*This currently fails on TensorFlow.jl*):

In [1]:
# !!! Restart your Julia kernel here !!!
using TensorFlow

# Create some variables.
@tf v1 = get_variable([3], Float64; initializer=ConstantInitializer(0.0))
@tf v2 = get_variable([5], Float64; initializer=ConstantInitializer(0.0))

# Add ops to save and restore only `v2`
saver = train.Saver(;var_list=[v2])

# Use the saver object normally after that.
sess = Session()
# Initialize v1 since the saver will not.
run(sess, v1.assign_node)
train.restore(saver, sess, "model.jld")
println("v1 : ", run(sess, v1))
println("v2 : ", run(sess, v2))

2018-10-05 09:48:14.780351: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA


KeyError: KeyError: key "v1" not found

Notes:

*  You can create as many `Saver` objects as you want if you need to save and
   restore different subsets of the model variables.  The same variable can be
   listed in multiple saver objects; its value is only changed when the
   `restore` method is run.

*  If you only restore a subset of the model variables at the start of a
   session, you have to run an initialize op for the other variables.

*  The ability to inspect variables in a checkpoint and save variables under a different name is not yet supported by TensorFlow.jl

## Build and load a SavedModel

This functionality is not yet supported by TensorFlow.jl

----------------------
*Except as otherwise noted, the content of this page is licensed under the [Creative Commons Attribution 3.0 License](https://creativecommons.org/licenses/by/3.0/), and code samples are licensed under the [Apache 2.0 License](https://www.apache.org/licenses/LICENSE-2.0). This notebook is adapted from the official [TensorFlow Guide on Save and Restore](https://www.tensorflow.org/guide/saved_model).*